Create Azure SQL Database
============================================

Steps of this procedure include:
1. Set variables and set up Notebook
1. Connect to Azure account and subscription
1. Provision firewall rules
1. Create SQL database resource

## Set variables
These variables are set based on your inputs in the deployment wizard. You can make changes to these variables but be aware of possible validation errors caused by your changes.






In [10]:
azure_sqldb_subscription = '016bc022-0dd3-4c1e-b2fd-299c6b026639'
azure_sqldb_resource_group_name = 'Group1'
azure_sqldb_server_name = 'hbdetect'
azure_sqldb_database_name = 'HeartbeatDetect'
azure_sqldb_collation = 'SQL_Latin1_General_CP1_CI_AS'
azure_sqldb_enable_firewall_rule = True
azure_sqldb_ip_start = '134.226.251.100'
azure_sqldb_ip_end = '134.226.251.200'
azure_sqldb_firewall_name = 'group14'


## Notebook Setup 

In [11]:
import sys, os, json, time, string, random, subprocess
def run_command(command, json_decode = True, printOutput = True):
    print(command)
    process = subprocess.Popen(command.split(), stdout=subprocess.PIPE, stderr=subprocess.PIPE, shell=True)
    output, error = process.communicate()
    if process.returncode != 0: 
        print("Process failed %d \n%s" % (process.returncode, error.decode("utf-8")))
        raise Exception()
    if output:
        output = output.decode("utf-8")
        if printOutput:
            print(output)
        try:
            return json.loads(output)
        except:
            return output


## Connecting to your Azure account


In [12]:
subscriptions = run_command('az account list', printOutput = False)
if azure_sqldb_subscription not in (subscription["id"] for subscription in subscriptions):
    run_command('az login')

az account list


## Setting your Azure subscription


In [13]:
run_command(
    'az account set '
    '--subscription {0}'
    .format(
        azure_sqldb_subscription));

az account set --subscription 016bc022-0dd3-4c1e-b2fd-299c6b026639


## Create a server firewall rule

This firewall rule will allow you to access your server and database within IP range immediately after it is created.

In [14]:
if azure_sqldb_enable_firewall_rule == True:
    create_firewall_rule_result = run_command(
        'az sql server firewall-rule create '
        '--start-ip-address {0} '
        '--end-ip-address {1} '
        '--server {2} '
        '--name {3} '
        '--resource-group {4} '
        .format(
            azure_sqldb_ip_start, 
            azure_sqldb_ip_end, 
            azure_sqldb_server_name, 
            azure_sqldb_firewall_name, 
            azure_sqldb_resource_group_name));

az sql server firewall-rule create --start-ip-address 134.226.251.100 --end-ip-address 134.226.251.200 --server hbdetect --name group14 --resource-group Group1 


{
  "endIpAddress": "134.226.251.200",
  "id": "/subscriptions/016bc022-0dd3-4c1e-b2fd-299c6b026639/resourceGroups/Group1/providers/Microsoft.Sql/servers/hbdetect/firewallRules/group14",
  "name": "group14",
  "resourceGroup": "Group1",
  "startIpAddress": "134.226.251.100",
  "type": "Microsoft.Sql/servers/firewallRules"
}



## Create Azure SQL Database

Learn more about the different [cost and performance options](https://go.microsoft.com/fwlink/?linkid=842067) and [other additional customizations](https://go.microsoft.com/fwlink/?linkid=2147212) for creating the database

In [15]:
create_database_result = run_command(
    'az sql db create '
    '--server {0} '
    '--name {1} '
    '--edition GeneralPurpose '
    '--compute-model Serverless '
    '--family Gen5 '
    '--resource-group {2} '
    '--min-capacity 0.5 '
    '--max-size 32GB '
    '--capacity 1 '
    '--collation {3} '
    .format(
        azure_sqldb_server_name, 
        azure_sqldb_database_name, 
        azure_sqldb_resource_group_name, 
        azure_sqldb_collation));

az sql db create --server hbdetect --name HeartbeatDetect --edition GeneralPurpose --compute-model Serverless --family Gen5 --resource-group Group1 --min-capacity 0.5 --max-size 32GB --capacity 1 --collation SQL_Latin1_General_CP1_CI_AS 


{
  "autoPauseDelay": 60,
  "catalogCollation": "SQL_Latin1_General_CP1_CI_AS",
  "collation": "SQL_Latin1_General_CP1_CI_AS",
  "createMode": null,
  "creationDate": "2022-03-21T15:36:42.213000+00:00",
  "currentBackupStorageRedundancy": "Geo",
  "currentServiceObjectiveName": "GP_S_Gen5_1",
  "currentSku": {
    "capacity": 1,
    "family": "Gen5",
    "name": "GP_S_Gen5",
    "size": null,
    "tier": "GeneralPurpose"
  },
  "databaseId": "07ca374a-924d-427f-b1cf-55213c06cfef",
  "defaultSecondaryLocation": "swedensouth",
  "earliestRestoreDate": null,
  "edition": "GeneralPurpose",
  "elasticPoolId": null,
  "elasticPoolName": null,
  "failoverGroupId": null,
  "highAvailabilityReplicaCount": null,
  "id": "/subscriptions/016bc022-0dd3-4c1e-b2fd-299c6b026639/resourceGroups/Group1/providers/Microsoft.Sql/servers/hbdetect/databases/HeartbeatDetect",
  "isInfraEncryptionEnabled": false,
  "kind": "v12.0,user,vcore,serverless",
  "ledgerOn": false,
  "licenseType": null,
  "location": 